In [17]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

In [15]:
CLIENT_ID = 'Y4KMS3PAB54UJZYMXWD3FT05P04QQJ15HFQJNH3Q4YJGKZJM' 
CLIENT_SECRET = 'OU1M4WVB1XD50GXWYSSYZ0AJTE3QZUJGT4ZXEN3ZZZUCDX10'
VERSION = '20200418'
LIMIT= 50

In [18]:
city = 'Villa de Leyva'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

5.6336805 -73.523548


In [19]:
search_query = 'Hotel'
radius = 500

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

result_hotel = requests.get(url).json()

result_hotel

venue_hotel = result_hotel['response']['venues']

df = json_normalize(venue_hotel)
df.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4e3cb9c7ae604542365c7d1f,Calle 8 # 10-35,CO,Villa de Leiva,Colombia,NaN,496,"[Calle 8 # 10-35, Villa De Leyva, Boyacá, Colo...","[{'label': 'display', 'lat': 5.633063855150655...",5.633064,-73.527990,NaN,Boyacá,Hotel & Spa Getsemaní,v-1587172300
1,"[{'id': '4bf58dd8d48988d1ed941735', 'name': 'S...",False,4f526baae4b023845e5fdd2f,Calle 8 # 10-35,CO,Villa de Leiva,Colombia,NaN,497,"[Calle 8 # 10-35, Villa De Leyva, Boyacá, Colo...","[{'label': 'display', 'lat': 5.633199593051142...",5.633200,-73.528011,NaN,Boyacá,Elea Spa - Hotel & Spa Getsemaní,v-1587172300
2,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4ebf521993ad36d7a8e101d0,Calle 10 7-31,CO,Villa de Leiva,Colombia,NaN,87,"[Calle 10 7-31, Villa de Leyva, Boyacá, Boyacá...","[{'label': 'display', 'lat': 5.634087665696955...",5.634088,-73.524222,NaN,Boyacá,Hotel Plaza Mayor,v-1587172300
3,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4cf3f52588de37044318822b,Calle 8,CO,Villa de Lleyva,Colombia,Carrera 11,159,"[Calle 8 (Carrera 11), Villa de Lleyva, Boyacá...","[{'label': 'display', 'lat': 5.63236, 'lng': -...",5.632360,-73.522994,NaN,Boyacá,Hotel San Antonio,v-1587172300
4,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",False,4e0fa8f37d8bb178a8b5221d,NaN,CO,NaN,Colombia,NaN,239,"[Boyacá, Colombia]","[{'label': 'display', 'lat': 5.634973593050659...",5.634974,-73.521820,NaN,Boyacá,Hotel Mesón de los Virreyes,v-1587172300


In [20]:
new_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')]+ ['id']
new_df = df.loc[:,new_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

new_df['categories'] = new_df.apply(get_category_type, axis=1)

new_df.columns = [column.split('.')[-1] for column in new_df.columns]

new_df.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Hotel & Spa Getsemaní,Hotel,Calle 8 # 10-35,CO,Villa de Leiva,Colombia,NaN,496,"[Calle 8 # 10-35, Villa De Leyva, Boyacá, Colo...","[{'label': 'display', 'lat': 5.633063855150655...",5.633064,-73.527990,NaN,Boyacá,4e3cb9c7ae604542365c7d1f
1,Elea Spa - Hotel & Spa Getsemaní,Spa,Calle 8 # 10-35,CO,Villa de Leiva,Colombia,NaN,497,"[Calle 8 # 10-35, Villa De Leyva, Boyacá, Colo...","[{'label': 'display', 'lat': 5.633199593051142...",5.633200,-73.528011,NaN,Boyacá,4f526baae4b023845e5fdd2f
2,Hotel Plaza Mayor,Hotel,Calle 10 7-31,CO,Villa de Leiva,Colombia,NaN,87,"[Calle 10 7-31, Villa de Leyva, Boyacá, Boyacá...","[{'label': 'display', 'lat': 5.634087665696955...",5.634088,-73.524222,NaN,Boyacá,4ebf521993ad36d7a8e101d0
3,Hotel San Antonio,Hotel,Calle 8,CO,Villa de Lleyva,Colombia,Carrera 11,159,"[Calle 8 (Carrera 11), Villa de Lleyva, Boyacá...","[{'label': 'display', 'lat': 5.63236, 'lng': -...",5.632360,-73.522994,NaN,Boyacá,4cf3f52588de37044318822b
4,Hotel Mesón de los Virreyes,Hotel,NaN,CO,NaN,Colombia,NaN,239,"[Boyacá, Colombia]","[{'label': 'display', 'lat': 5.634973593050659...",5.634974,-73.521820,NaN,Boyacá,4e0fa8f37d8bb178a8b5221d


In [21]:
new_df= new_df.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id'], axis=1)
new_df

,name,categories,address,lat,lng,postalCode,state
0,Hotel & Spa Getsemaní,Hotel,Calle 8 # 10-35,5.633064,-73.527990,NaN,Boyacá
1,Elea Spa - Hotel & Spa Getsemaní,Spa,Calle 8 # 10-35,5.633200,-73.528011,NaN,Boyacá
2,Hotel Plaza Mayor,Hotel,Calle 10 7-31,5.634088,-73.524222,NaN,Boyacá
3,Hotel San Antonio,Hotel,Calle 8,5.632360,-73.522994,NaN,Boyacá
4,Hotel Mesón de los Virreyes,Hotel,NaN,5.634974,-73.521820,NaN,Boyacá
5,hotel Y centro de convenciones casa de los fun...,Hotel,1km Via Guanani,5.633522,-73.524962,NaN,Boyacá
6,Hotel Boutique La Espanola,Hotel,Calle 12 No. 11 - 06,5.634637,-73.525069,NaN,Boyacá
7,Hotel Villa Paz,Ski Lodge,Crr 11a no. 12 - 27,5.635055,-73.525024,154001,Boyacá
8,Hotel Casa Cantabria,Bed & Breakfast,NaN,5.632814,-73.525732,NaN,Boyacá
9,Hotel Antonio Nariño,Bed & Breakfast,Villa De Leyva,5.631885,-73.524600,NaN,Boyacá


In [22]:


array= ['Hotel']
df_hotel= new_df.loc[new_df['categories'].isin(array)]
df_hotel



,name,categories,address,lat,lng,postalCode,state
0,Hotel & Spa Getsemaní,Hotel,Calle 8 # 10-35,5.633064,-73.527990,NaN,Boyacá
2,Hotel Plaza Mayor,Hotel,Calle 10 7-31,5.634088,-73.524222,NaN,Boyacá
3,Hotel San Antonio,Hotel,Calle 8,5.632360,-73.522994,NaN,Boyacá
4,Hotel Mesón de los Virreyes,Hotel,NaN,5.634974,-73.521820,NaN,Boyacá
5,hotel Y centro de convenciones casa de los fun...,Hotel,1km Via Guanani,5.633522,-73.524962,NaN,Boyacá
6,Hotel Boutique La Espanola,Hotel,Calle 12 No. 11 - 06,5.634637,-73.525069,NaN,Boyacá
10,Hotel Plazuela de San Agustín,Hotel,NaN,5.634718,-73.521240,NaN,Boyacá
12,Hotel Aqua Vitae,Hotel,NaN,5.634297,-73.528244,154001,Boyacá
13,Hotel Campanario de la Villa,Hotel,NaN,5.635248,-73.525080,NaN,NaN
14,Hotel Miracle,Hotel,Carrera 11 #9-41,5.633403,-73.526484,NaN,Boyacá


In [27]:
search_query = 'Restaurant'
radius = 10000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=Y4KMS3PAB54UJZYMXWD3FT05P04QQJ15HFQJNH3Q4YJGKZJM&client_secret=OU1M4WVB1XD50GXWYSSYZ0AJTE3QZUJGT4ZXEN3ZZZUCDX10&ll=5.6336805,-73.523548&v=20200418&query=Restaurant&radius=10000&limit=50'

In [28]:
result_restaurant = requests.get(url).json()

venue_restaurant = result_restaurant['response']['venues']

Restaurant_df = json_normalize(venue_restaurant)
Restaurant_df.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",False,4f370d08e4b0d3ab511ef894,Villa de leyva,CO,NaN,Colombia,NaN,33,"[Villa de leyva, Colombia]","[{'label': 'display', 'lat': 5.633776342358105...",5.633776,-73.523260,NaN,NaN,NaN,Restaurante Portales,v-1587172503,NaN
1,"[{'id': '52e81612bcbc57f1066b7a00', 'name': 'C...",False,534ead7d498e662b28a28cbd,NaN,CO,NaN,Colombia,NaN,32,[Colombia],"[{'label': 'display', 'lat': 5.633417588346402...",5.633418,-73.523680,NaN,NaN,NaN,El Santo Restaurante Gourmet,v-1587172503,NaN
2,"[{'id': '4bf58dd8d48988d10c941735', 'name': 'F...",False,4ef607e20e61a0846c52c3d6,NaN,CO,Villa de Leiva,Colombia,NaN,101,"[Villa de Leyva, Boyacá, Colombia]","[{'label': 'display', 'lat': 5.634233089432019...",5.634233,-73.522817,NaN,NaN,Boyacá,Restaurante Chez Rémy,v-1587172503,NaN
3,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",False,50edd8a9e4b037265a58382e,Cr10 No 11-50,CO,Villa de Leiva,Colombia,NaN,130,"[Cr10 No 11-50, Villa de Leyva, Boyacá, Colombia]","[{'label': 'display', 'lat': 5.633534154682136...",5.633534,-73.524718,Centro Historico,154001,Boyacá,Restaurante Carnes y Olivas,v-1587172503,NaN
4,"[{'id': '4bf58dd8d48988d10c941735', 'name': 'F...",False,4e8798c630f83936723c7bbc,Carrera 9 13-51,CO,Villa de Leiva,Colombia,NaN,120,"[Carrera 9 13-51, Villa de Leyva, Boyacá, Colo...","[{'label': 'display', 'lat': 5.634215967092749...",5.634216,-73.522599,NaN,154001,Boyacá,Restaurante Mama Santa,v-1587172503,NaN


In [29]:
Restaurant_new_columns = ['name', 'categories'] + [col for col in Restaurant_df.columns if col.startswith('location.')]+ ['id']
new_df_restaurant = Restaurant_df.loc[:,Restaurant_new_columns]

def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

new_df_restaurant['categories'] = new_df_restaurant.apply(get_category_type, axis=1)

new_df_restaurant.columns = [column.split('.')[-1] for column in new_df_restaurant.columns]

new_df_restaurant.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Restaurante Portales,Pizza Place,Villa de leyva,CO,NaN,Colombia,NaN,33,"[Villa de leyva, Colombia]","[{'label': 'display', 'lat': 5.633776342358105...",5.633776,-73.523260,NaN,NaN,NaN,4f370d08e4b0d3ab511ef894
1,El Santo Restaurante Gourmet,Comfort Food Restaurant,NaN,CO,NaN,Colombia,NaN,32,[Colombia],"[{'label': 'display', 'lat': 5.633417588346402...",5.633418,-73.523680,NaN,NaN,NaN,534ead7d498e662b28a28cbd
2,Restaurante Chez Rémy,French Restaurant,NaN,CO,Villa de Leiva,Colombia,NaN,101,"[Villa de Leyva, Boyacá, Colombia]","[{'label': 'display', 'lat': 5.634233089432019...",5.634233,-73.522817,NaN,NaN,Boyacá,4ef607e20e61a0846c52c3d6
3,Restaurante Carnes y Olivas,Restaurant,Cr10 No 11-50,CO,Villa de Leiva,Colombia,NaN,130,"[Cr10 No 11-50, Villa de Leyva, Boyacá, Colombia]","[{'label': 'display', 'lat': 5.633534154682136...",5.633534,-73.524718,Centro Historico,154001,Boyacá,50edd8a9e4b037265a58382e
4,Restaurante Mama Santa,French Restaurant,Carrera 9 13-51,CO,Villa de Leiva,Colombia,NaN,120,"[Carrera 9 13-51, Villa de Leyva, Boyacá, Colo...","[{'label': 'display', 'lat': 5.634215967092749...",5.634216,-73.522599,NaN,154001,Boyacá,4e8798c630f83936723c7bbc


In [30]:


new_df_restaurant= new_df_restaurant.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'id'], axis=1)
new_df_restaurant



,name,categories,address,lat,lng,postalCode,state
0,Restaurante Portales,Pizza Place,Villa de leyva,5.633776,-73.523260,NaN,NaN
1,El Santo Restaurante Gourmet,Comfort Food Restaurant,NaN,5.633418,-73.523680,NaN,NaN
2,Restaurante Chez Rémy,French Restaurant,NaN,5.634233,-73.522817,NaN,Boyacá
3,Restaurante Carnes y Olivas,Restaurant,Cr10 No 11-50,5.633534,-73.524718,154001,Boyacá
4,Restaurante Mama Santa,French Restaurant,Carrera 9 13-51,5.634216,-73.522599,154001,Boyacá
5,Los Portales Restaurante,Restaurant,NaN,5.633803,-73.523068,NaN,NaN
6,Restaurante El Caney,Restaurant,NaN,5.634393,-73.521957,NaN,NaN
7,Restaurante Peruano del Carajo,Peruvian Restaurant,NaN,5.634674,-73.525313,NaN,NaN
8,La Romana Restaurante y Pizzeria,Pizza Place,Calle 12# 9- 67,5.633489,-73.524030,NaN,Boyacá
9,Savia Orgánica - Restaurante,Restaurant,NaN,5.633183,-73.523708,NaN,NaN


In [31]:
df_Restaurant = new_df_restaurant.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_Restaurant

,name,categories,address,lat,lng,postalCode,state
3,Restaurante Carnes y Olivas,Restaurant,Cr10 No 11-50,5.633534,-73.524718,154001,Boyacá
4,Restaurante Mama Santa,French Restaurant,Carrera 9 13-51,5.634216,-73.522599,154001,Boyacá
17,Arcadia Bar Restaurante,Italian Restaurant,calle 13 9-10,5.633889,-73.522909,154001,Boyacá
18,Restaurante Tabú Cocina & Arte,Fast Food Restaurant,Cra 7 N 7b - 49,5.629348,-73.525384,154001,Boyacá


In [32]:
neighbourhood_df = pd.concat([df_hotel, df_Restaurant], ignore_index=True)
neighbourhood_df

,name,categories,address,lat,lng,postalCode,state
0,Hotel & Spa Getsemaní,Hotel,Calle 8 # 10-35,5.633064,-73.527990,NaN,Boyacá
1,Hotel Plaza Mayor,Hotel,Calle 10 7-31,5.634088,-73.524222,NaN,Boyacá
2,Hotel San Antonio,Hotel,Calle 8,5.632360,-73.522994,NaN,Boyacá
3,Hotel Mesón de los Virreyes,Hotel,NaN,5.634974,-73.521820,NaN,Boyacá
4,hotel Y centro de convenciones casa de los fun...,Hotel,1km Via Guanani,5.633522,-73.524962,NaN,Boyacá
5,Hotel Boutique La Espanola,Hotel,Calle 12 No. 11 - 06,5.634637,-73.525069,NaN,Boyacá
6,Hotel Plazuela de San Agustín,Hotel,NaN,5.634718,-73.521240,NaN,Boyacá
7,Hotel Aqua Vitae,Hotel,NaN,5.634297,-73.528244,154001,Boyacá
8,Hotel Campanario de la Villa,Hotel,NaN,5.635248,-73.525080,NaN,NaN
9,Hotel Miracle,Hotel,Carrera 11 #9-41,5.633403,-73.526484,NaN,Boyacá


In [ ]:


map_villa_de_leyva = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(neighbourhood_df['lat'], neighbourhood_df['lng'], 
                                           neighbourhood_df['name'], neighbourhood_df['categories'],\
                                               neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_villa_de_leyva